In [1]:
import torch
import wandb
from omegaconf import OmegaConf

In [2]:
model = "Siren"

sweep_config = OmegaConf.to_container(OmegaConf.load(f'./configs/{model}_INR_sweep_cfg.yaml'))

In [3]:
from INR import RFF, MLP, Gabor
from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor
from INR.utils import cifar_grid

training_set = CIFAR10(transform=ToTensor(), root="data", download=True)
def get_training_imgs(batch_size, device, image_idx):
    training_imgs = []
    input_imgs = []
    for image_id in image_idx:
        training_img = training_set.__getitem__(image_id)[0].permute((1,2,0))
        training_imgs.append(training_img.to(device))
    training_imgs = torch.stack(training_imgs, 0)
    return training_imgs

def make_RFF(domain, codomain, n_hidden, hidden_size, batch_size, sigma, device, learning_rate, lr_factor, epochs):
    model = RFF(domain=domain, codomain=codomain, hidden_features=[hidden_size]*n_hidden, batch_size=batch_size,
                sigma=sigma).to(device)
    optimizer = torch.optim.Adam(model.model.parameters(), lr=learning_rate)
    criterion = torch.nn.MSELoss(reduction='mean')
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=lr_factor, patience=1)
    training_imgs = get_training_imgs(batch_size, device, list(range(batch_size)))
    train_coordinates = cifar_grid(32, batch_size).to(device)
    return model, optimizer, criterion, scheduler, training_imgs, train_coordinates, epochs

def make_MLP(domain, codomain, n_hidden, hidden_size, batch_size, activ_func, final_non_linearity, omega0, omega0_initial,
             device, learning_rate, lr_factor, epochs):
    model = MLP(domain=domain, codomain=codomain, hidden_size=[hidden_size]*n_hidden, batch_size=batch_size,
                activ_func=activ_func, final_non_linearity=final_non_linearity, omega0=omega0, omega0_initial=omega0_initial
               ).to(device)
    optimizer = torch.optim.Adam(model.model.parameters(), lr=learning_rate)
    criterion = torch.nn.MSELoss(reduction='mean')
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=lr_factor, patience=1)
    training_imgs = get_training_imgs(batch_size, device, list(range(batch_size)))
    train_coordinates = cifar_grid(32, batch_size).to(device)
    return model, optimizer, criterion, scheduler, training_imgs, train_coordinates, epochs

def make_Gabor(domain, codomain, no_layers, hidden_channels, batch_size, input_scale, weight_scale, alpha, beta, bias,
               init_spatial_value, covariance, final_non_linearity, device, learning_rate, lr_factor, epochs):
    model = Gabor(domain=domain, codomain=codomain, no_layers=no_layers, hidden_channels=hidden_channels, batch_size=batch_size,
                  input_scale=input_scale, weight_scale=weight_scale, alpha=alpha, beta=beta, bias=bias,
                  init_spatial_value=init_spatial_value, covariance=covariance, final_non_linearity=final_non_linearity
                 ).to(device)
    optimizer = torch.optim.Adam(model.model.parameters(), lr=learning_rate)
    criterion = torch.nn.MSELoss(reduction='mean')
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=lr_factor, patience=1)
    training_imgs = get_training_imgs(batch_size, device, list(range(batch_size)))
    train_coordinates = cifar_grid(32, batch_size).to(device)
    return model, optimizer, criterion, scheduler, training_imgs, train_coordinates, epochs

def train_RFF():
    with wandb.init() as run:
        config = wandb.config
        model, optimizer, criterion, scheduler, training_imgs, train_coordinates, epochs = make_RFF(**config)
        wandb.watch(model.model)
        model.fit(training_imgs, optimizer, criterion, scheduler, epochs, image_grid=train_coordinates, log=True)

def train_MLP():
    with wandb.init() as run:
        config = wandb.config
        model, optimizer, criterion, scheduler, training_imgs, train_coordinates, epochs = make_MLP(**config)
        wandb.watch(model.model)
        model.fit(training_imgs, optimizer, criterion, scheduler, epochs, image_grid=train_coordinates, log=True)
        
def train_Gabor():
    with wandb.init() as run:
        config = wandb.config
        model, optimizer, criterion, scheduler, training_imgs, train_coordinates, epochs = make_Gabor(**config)
        wandb.watch(model.model)
        model.fit(training_imgs, optimizer, criterion, scheduler, epochs, image_grid=train_coordinates, log=True)

Files already downloaded and verified


In [ ]:
train_fn = train_RFF if model == "RFF" else train_MLP if model == "Siren" or model == "GARF" else train_Gabor

sweep_id = wandb.sweep(sweep_config, project="INR", entity="rvanbakel")
wandb.agent(sweep_id, function=train_fn)

Create sweep with ID: zek2ff83
Sweep URL: https://wandb.ai/rvanbakel/INR/sweeps/zek2ff83


wandb: Agent Starting Run: 0pgj1kpv with config:
wandb: 	activ_func: sine
wandb: 	batch_size: 5
wandb: 	codomain: [3]
wandb: 	device: cuda
wandb: 	domain: 2
wandb: 	epochs: [200, 200, 200, 200, 200]
wandb: 	final_non_linearity: sigmoid
wandb: 	hidden_size: 30
wandb: 	learning_rate: 0.01
wandb: 	lr_factor: 0.8
wandb: 	n_hidden: 5
wandb: 	omega0: 9
wandb: 	omega0_initial: 0.03
wandb: Currently logged in as: rvanbakel (use `wandb login --relogin` to force relogin)


C:\Users\Gebruiker\anaconda3\envs\master_thesis\lib\site-packages\torch\functional.py:1069: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2157.)
  return _VF.cartesian_prod(tensors)  # type: ignore[attr-defined]


MSE,█▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
MSE (Upsampled),█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
MSE,0.00034
MSE (Upsampled),0.00509


wandb: Agent Starting Run: 1xbpmtt9 with config:
wandb: 	activ_func: sine
wandb: 	batch_size: 5
wandb: 	codomain: [3]
wandb: 	device: cuda
wandb: 	domain: 2
wandb: 	epochs: [200, 200, 200, 200, 200]
wandb: 	final_non_linearity: sigmoid
wandb: 	hidden_size: 30
wandb: 	learning_rate: 0.01
wandb: 	lr_factor: 0.8
wandb: 	n_hidden: 5
wandb: 	omega0: 9
wandb: 	omega0_initial: 0.035


...

wandb: Agent Starting Run: osddmeht with config:
wandb: 	activ_func: sine
wandb: 	batch_size: 5
wandb: 	codomain: [3]
wandb: 	device: cuda
wandb: 	domain: 2
wandb: 	epochs: [200, 200, 200, 200, 200]
wandb: 	final_non_linearity: sigmoid
wandb: 	hidden_size: 20
wandb: 	learning_rate: 0.01
wandb: 	lr_factor: 0.8
wandb: 	n_hidden: 4
wandb: 	omega0: 10
wandb: 	omega0_initial: 0.04


MSE,█▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
MSE (Upsampled),█▂▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂
MSE,0.00057
MSE (Upsampled),0.00497


wandb: Agent Starting Run: q5wr0pmd with config:
wandb: 	activ_func: sine
wandb: 	batch_size: 5
wandb: 	codomain: [3]
wandb: 	device: cuda
wandb: 	domain: 2
wandb: 	epochs: [200, 200, 200, 200, 200]
wandb: 	final_non_linearity: sigmoid
wandb: 	hidden_size: 20
wandb: 	learning_rate: 0.01
wandb: 	lr_factor: 0.8
wandb: 	n_hidden: 4
wandb: 	omega0: 11
wandb: 	omega0_initial: 0.03


MSE,█▂▂▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
MSE (Upsampled),█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁
MSE,0.00049
MSE (Upsampled),0.00515


wandb: Agent Starting Run: bohntm1n with config:
wandb: 	activ_func: sine
wandb: 	batch_size: 5
wandb: 	codomain: [3]
wandb: 	device: cuda
wandb: 	domain: 2
wandb: 	epochs: [200, 200, 200, 200, 200]
wandb: 	final_non_linearity: sigmoid
wandb: 	hidden_size: 20
wandb: 	learning_rate: 0.01
wandb: 	lr_factor: 0.8
wandb: 	n_hidden: 4
wandb: 	omega0: 11
wandb: 	omega0_initial: 0.035
